In [24]:
from time import sleep
import pandas as pd
import tkinter as tk
import requests
import numpy as np

In [4]:
def run_query(query):
    response = requests.post('https://api.tarkov.dev/graphql', json={'query': query})
    if response.status_code == 200:
        return response.json()
    else:
        raise Exception("Query failed to run by returning code of {}. {}".format(response.status_code, query))

# lists all items
query_all_items_prices = """
{
    items {
        shortName
        id
        width
        height
        avg24hPrice
        changeLast48hPercent
        sellFor {
            price
            source
        }
    }
}
"""

def getAllItemsPrices():
    result = run_query(query_all_items_prices)
    #result = run_query(water_query)
    return result['data']['items']

In [5]:
path_data = './data/'

In [6]:
def items_dict_to_df(all_items):
    all_items_df = pd.DataFrame(columns=['name', 'id', 'width', 'height', 'icon_width', 'icon_height', 'features', 'fleaMarket', 'flea_avg48', 'flea_ch48percent', 'prapor', 'therapist', 'fence', 'skier', 'peacekeeper', 'mechanic', 'ragman', 'jaeger'], index=range(len(all_items)))

    # iterate over all items
    for index in range(len(all_items)):
        all_items_df.iloc[index]['name'] = all_items[index].get('shortName')
        all_items_df.iloc[index]['id'] = all_items[index].get('id')
        all_items_df.iloc[index]['width'] = all_items[index].get('width')
        all_items_df.iloc[index]['height'] = all_items[index].get('height')
        all_items_df.iloc[index]['flea_avg48'] = all_items[index].get('avg24hPrice')
        all_items_df.iloc[index]['flea_ch48percent'] = all_items[index].get('changeLast48hPercent')

        # iterate over all traders that can buy the item
        for offer in all_items[index].get('sellFor'):
            trader = offer.get('source')
            price  = offer.get('price')
            all_items_df.iloc[index][trader] = price
    
    return all_items_df


In [7]:
# create the items dataframe
all_items_dict = getAllItemsPrices()

In [8]:
all_items_df   = items_dict_to_df(all_items_dict)

In [9]:
# save dataframe as csv
all_items_df.to_csv(path_data + 'all_items.csv', index=False)

OSError: Cannot save file into a non-existent directory: 'data'

In [ ]:
# load dataframe from csv
all_items_df = pd.read_csv(path_data + 'all_items.csv')

In [26]:
item

name                               6P20 0-20
id                  5649aa744bdc2ded0b8b457e
width                                      1
height                                     1
icon_width                               NaN
icon_height                              NaN
features                                 NaN
fleaMarket                              3000
flea_avg48                              6700
flea_ch48percent                      -41.16
prapor                                   NaN
therapist                                NaN
fence                                    277
skier                                    340
peacekeeper                                3
mechanic                                 388
ragman                                   NaN
jaeger                                   NaN
Name: 100, dtype: object

In [29]:
item = all_items_df.loc[100]
price_traders = item.loc['prapor':'jaeger']
price_max = np.nanmax(price_traders)
trader = np.nanargmax(price_traders)
trader

5

In [ ]:
# all_items_df

In [ ]:
# all_items_dict